<a href="https://colab.research.google.com/github/Varshitha-Sathish/repo_post/blob/main/Merger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import geopy.distance as gd
import numpy as np
import random

In [27]:

def haversine(lat1, lon1, lat2, lon2):
    # uses km. do not fuck with.


    R = 6371  # Earth radius in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate distance
    distance = R * c

    return distance

def distance(lat1, long1, lat2, long2):
  # haversine but miles though
  return haversine(lat1, long1, lat2, long2) * 0.621371

In [28]:
#FL = pd.read_excel('/content/2007_statewide/Florida.xlsx')
#IL = pd.read_excel('/content/2007_statewide/Illinois.xlsx')
#NYNJ = pd.read_excel('/content/2007_statewide/NJ-NY.xlsx')
#TX = pd.read_excel('/content/2007_statewide/Texas.xlsx')
ThreeCorners = pd.read_excel('/3 states.xlsx')
#upload whatever files you need here and assign the data value
data = ThreeCorners

# Creates copy of index called BANKID
data["BANKID"] = data.index.values.tolist()

numBanks = len(data)

In [29]:
# CLEARS DISTTABLE!

# distTable is a memoization table where distTable[A][B] is the distance from A to B
# where A and B are indices in df.



distTable = {}
#This line initializes an empty dictionary called distTable. In Python, a dictionary is a collection of key-value pairs.
# distTable = [[0 if i == j else None for j in range(numBanks)] for i in range(numBanks)]

In [30]:
# Search returns a dataframe of banks within radius miles of bank specified by bankID.
#This is a function definition. It defines a function named search that takes three parameters: bankID (an integer), radius (a floating-point number), and raw.
def search(bankID:int, radius:float, raw):

  if bankID >= numBanks or bankID < 0:
    print("Error: invalid BankID for this dataset!")
    return None

  # get coordinates of bank with bankID- This extracts the latitude and longitude of the specified bank from the raw data.
  lat = raw['LATITUDE'][bankID]
  lon = raw['LONGITUDE'][bankID]

  # DATA VALIDATION: only keeps banks with valid LONGITUDE and LATITUDE
  df_valid = raw.dropna(axis='index', subset=['LATITUDE','LONGITUDE'])
  # TODO: query backup longitude & latitude?


  # shortcut: will not consider banks outside of a square with side length = radius. 1 degree=69.4 miles, we don't go towards the north pole so we dc
  #This filters the df_valid DataFrame to only include banks within a square region (of side length radius) centered around the specified bank.
  #It then drops the row corresponding to the specified bank.
  local = df_valid[(abs(df_valid['LATITUDE'] - lat) < (radius/69.4)) & (abs(df_valid['LONGITUDE'] - lon) < (radius/69.4))].drop(index=bankID)

  # for each branch: This sets up a loop to iterate over each bank in the local DataFrame.
  for b in local.index:
    d = None

    # if the distance has already been computed, just use that!
    if bankID in distTable and b in distTable[bankID]:
      d = distTable[bankID][b]
    # otherwise, compute it now:
    else:
      d = distance(raw['LATITUDE'][bankID], raw['LONGITUDE'][bankID], local['LATITUDE'][b], local['LONGITUDE'][b])
      # then, make sure it is stored properly...
      if bankID in distTable:
        distTable[bankID][b] = d
      else:
        distTable[bankID] = {b: d}
      # ...along with its inverse!
      if b in distTable:
        distTable[b][bankID] = d
      else:
        distTable[b] = {bankID: d}

#Check if the distance is already computed:
#It checks if bankID exists in distTable (the memoization table) and if b exists in distTable[bankID]. If both conditions are true, it means the distance has been computed before, so it retrieves the distance d from the memoization table.
#If the distance is not computed:
#If the conditions in the first part are not met (i.e., the distance is not already computed), it calculates the distance using the distance function.
#Store the computed distance in the memoization table:
#After computing the distance, it stores the result in the memoization table (distTable). It checks if bankID is already in distTable and updates the entry for bankID with the distance d. If bankID is not present, it creates a new entry in distTable for bankID with the distance d. The same is done for the inverse relationship (b to bankID).

#This checks if the distance (d) is greater than the specified radius. If yes, it drops the current bank (b) from the local DataFrame.
    if d > radius:
      local = local.drop(index=b)

  # testing statement: lists max distance of query results
  # maxDist = None
  # for b in local.index:
  #   maxDist = max(distTable[bankID][b], maxDist)
  # print("Max Distance:", maxDist)
  # calculates and prints the maximum distance among the query results.

  return local
  #Finally, the function returns the filtered DataFrame containing banks within the specified radius of the specified bank.

In [31]:
#This Python function, multiSearch, appears to perform a search for nearby branches of a specified bank within a given radius.

def multiSearch(bankName:str, radius:float, raw):

#This line filters the raw DataFrame to include only rows where the "NAMEFULL" column matches the specified bankName.
#This creates a subset of data containing information about branches of the specified bank.

  companySet = raw[raw["NAMEFULL"] == bankName] #.astype({'UNINUMBR': 'int32'}).set_index("UNINUMBR")
  print("Exploring near", len(companySet), "branches of", bankName + "...")

#Initializes an empty dictionary to store the search results.
  results = {}

#Iterates over the indices (bank IDs) of the companySet DataFrame, representing branches of the specified bank.
  for bankID in companySet.index:
#Calls the search function for each bank in companySet. This function is expected to return a DataFrame of nearby branches.
    result = search(bankID, radius, raw)
#Adds the result of the search to the results dictionary.
#If the search result is empty (length is 0), it assigns None to the dictionary entry; otherwise, it assigns the search result DataFrame.
    results[bankID] = None if len(result) == 0 else result

  print("Search within", radius, "mile radius is complete!")
#Randomly selects one result from the results dictionary and prints it. This is done using the random.choice function.
  sampleID, sample = random.choice(list(results.items()))

  print("Random result (bankID = " + str(sampleID) + "): ")
  display(sample)

  return results

In [49]:
JPMorganChase = "JPMorgan Chase Bank, National Association"

JPQUERY = multiSearch(JPMorganChase, 5.0, data)

In [48]:
WashingtonMutual = "Washington Mutual Bank"

WMQUERY = multiSearch(WashingtonMutual, 5.0, data)


In [42]:
import pandas as pd

# Define the multiSearch function (replace this with the actual implementation)
def multiSearch(query, threshold, data):
    # Your implementation here
    pass

# Assuming data is a list or any other iterable
data = [...]  # Replace [...] with your actual data

# Assuming WMQUERY is a list of data
WashingtonMutual = "Washington Mutual Bank"
WMQUERY = multiSearch(WashingtonMutual, 5.0, data)

# Convert the list to a DataFrame
df = pd.DataFrame(WMQUERY, columns=["Column1", "Column2", ...])  # Replace "Column1", "Column2", ... with your actual column names

# Specify the Excel file path
excel_file_path = "output2.xlsx"

# Export the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f"Excel file saved to {excel_file_path}")


Excel file saved to output2.xlsx


In [41]:

# this one isn't working because I don't have access to the terminal!
# But if you ran it on the native machine, it should work.
def interface():

  bankName = input("Please enter the FULL NAME of the banking company you would like to search around: ")

  if bankName == "JP": bankName = "JPMorgan Chase Bank, National Association"
  if bankName == "WM": bankName = "Washington Mutual Bank"

  radius = float(input("Enter the search radius in miles: "))

  results = multiSearch(bankName, radius, data)

  return results



In [ ]:
from google.colab import drive
drive.mount('/content/drive')